# Creating command-line Tools

In [1]:
from ctapipe.core import Tool, Component, TelescopeComponent
from ctapipe.core.traits import (
    Integer, Float, List, Dict, Unicode,
    TraitError, observe, FloatTelescopeParameter,
    Path,
)
import logging
from time import sleep
from astropy import units as u
from ctapipe.utils import get_dataset_path

/usr/local/lib/python3.8/site-packages/setuptools_scm/git.py:68: UserWarning: "/github/workspace" is shallow and may cause errors
  warnings.warn('"{}" is shallow and may cause errors'.format(wd.path))


In [2]:
GAMMA_FILE = get_dataset_path('gamma_test.simtel.gz')

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

## Setup:

Create a few `Component`s that we will use later in a `Tool`:

In [3]:
class MyComponent(Component):
    """ A Component that does stuff """

    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")


# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    """ An advanced technique """

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Path(
        help="input file name", 
        exists=None,  # set to True to require existing, False for requiring non-existing
        directory_ok=False,
    ).tag(config=True)
    outfile = Path(
        help="output file name",
        exists=False, directory_ok=False
    ).tag(config=True)
    
    def __init__(self, config=None, parent=None, **kwargs):
        super().__init__(config=config, parent=parent, **kwargs)
        # components can have sub components, but these must have 
        # then parent=self as argument and be assigned as member
        # so the full config can be received later
        self.subcompent = MyComponent(parent=self)

    @observe("outfile")
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))


class TelescopeWiseComponent(TelescopeComponent):
    """ a component that contains parameters that are per-telescope configurable """

    param = FloatTelescopeParameter(
        help="Something configurable with telescope patterns", default_value=5.0
    ).tag(config=True)

In [4]:
MyComponent()

value,-1,Value to use (default: -1)


In [5]:
AdvancedComponent()

infile,None,input file name (default: None)
outfile,None,output file name (default: None)
value1,-1,Value to use (default: -1)


`TelescopeComponents` need to have a subarray given to them in order to work (since they need one to turn a `TelescopeParameter` into a concrete list of values for each telescope.  Here we will give a dummy one:

In [6]:
from ctapipe.instrument import SubarrayDescription, TelescopeDescription

subarray = SubarrayDescription(
    "Junk",
    tel_positions={1: (0.0, 0.0, 0.0) * u.m, 2: (1.0, 1.0, 0.0) * u.m},
    tel_descriptions={
        1: TelescopeDescription.from_name("LST", "LSTCam"),
        2: TelescopeDescription.from_name("MST", "NectarCam"),
    },
)
subarray.info()

Subarray : Junk
Num Tels : 2
Footprint: 0.00 km2

       Type       Count Tel IDs
----------------- ----- -------
   LST_LST_LSTCam     1 1      
MST_MST_NectarCam     1 2      


In [7]:
TelescopeWiseComponent(subarray=subarray)

param,"[('type', '*', 5.0)]",Something configurable with telescope patterns (default: traitlets.Undefined)


This TelescopeParameters can then be set using a list of patterns like:
```python
component.param = [ 
    ("type", "LST*",3.0), 
    ("type", "MST*", 2.0), 
    (id, 25, 4.0) 
]
```

These get translated into per-telescope-id values once the subarray is registered.  After that one acccess the per-telescope id values via:
```python
component.param.tel[tel_id]
```

## Now create an executable Tool that contains the Components

In [8]:
class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = dict(
        infile='AdvancedComponent.infile',
        iterations='MyTool.iterations'
    )

    # Which classes are registered for configuration
    classes = List([MyComponent, AdvancedComponent, SecondaryMyComponent, TelescopeWiseComponent])

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def setup(self):
        self.comp = MyComponent(parent=self)
        self.comp2 = SecondaryMyComponent(parent=self)
        self.comp3 = TelescopeWiseComponent(parent=self, subarray=subarray)
        self.advanced = AdvancedComponent(parent=self)


    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.1)
            
    def finish(self):
        self.log.warning("Shutting down.")
    

## Get Help info

The following allows you to print the help info within a Jupyter notebook, but this same inforamtion would be displayed if the user types:
```
  mytool --help
```

In [9]:
tool=MyTool()

In [10]:
tool.print_help()

do some things and stuff

Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    Set log-level to debug, for the most verbose logging.
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
-q
    Disable console logging.
    Equivalent to: [--Tool.quiet=True]
--no-q
    set Tool.quiet=False
    Equivalent to: [--Tool.quiet=False]
--infile=<Path>
    input file name
    Default: None
    Equivalent to: [--AdvancedComponent.infile]
--iterations=<Int>
    Number of times to run
    Default: 5
    Equivalent to: [--MyTool.iterations]
--config=<Path>
    nam

The following  is equivalant to the user typing `mytool --help-all`

In [11]:
tool.print_help(classes=True)

do some things and stuff

Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    Set log-level to debug, for the most verbose logging.
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
-q
    Disable console logging.
    Equivalent to: [--Tool.quiet=True]
--no-q
    set Tool.quiet=False
    Equivalent to: [--Tool.quiet=False]
--infile=<Path>
    input file name
    Default: None
    Equivalent to: [--AdvancedComponent.infile]
--iterations=<Int>
    Number of times to run
    Default: 5
    Equivalent to: [--MyTool.iterations]
--config=<Path>
    nam

## Run the tool

here we pass in argv since it is a Notebook, but if argv is not specified it's read from `sys.argv`, so the following is the same as running:

```sh
mytool --log_level=INFO --infile gamma_test.simtel.gz --iterations=3
```

As Tools are intended to be exectutables, they are raising `SystemExit` on exit.
Here, we use them to demonstrate how it would work, so we catch the `SystemExit`.

In [12]:
try:
    tool.run(argv=['--infile',  str(GAMMA_FILE)])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2021-02-09 13:50:28,306 WARNING [ctapipe.mytool] (<ipython-input-1-d7fc398e7046>.finish): Shutting down.


In [13]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 


try:
    tool.run(argv=['--log-level','INFO','--infile', str(GAMMA_FILE), '--iterations','3'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

/usr/local/lib/python3.8/site-packages/traitlets/config/application.py:204: RuntimeWarning: No Handler found on <Logger ctapipe.mytool (WARNING)>, setting log_format will have no effect
  warnings.warn(


2021-02-09 13:50:28,330 INFO [ctapipe.mytool] (tool.initialize): ctapipe version 0.1.dev1+gceb4210


2021-02-09 13:50:28,331 INFO [ctapipe.mytool] (tool.run): Starting: mytool


2021-02-09 13:50:28,344 INFO [ctapipe.mytool] (<ipython-input-1-d7fc398e7046>.start): Performing 3 iterations...


2021-02-09 13:50:28,345 INFO [ctapipe.mytool] (<ipython-input-1-d7fc398e7046>.start): ITERATION 0


...

2021-02-09 13:50:28,547 INFO [ctapipe.mytool] (<ipython-input-1-d7fc398e7046>.start): ITERATION 2


2021-02-09 13:50:28,648 WARNING [ctapipe.mytool] (<ipython-input-1-d7fc398e7046>.finish): Shutting down.


2021-02-09 13:50:28,648 INFO [ctapipe.mytool] (tool.run): Finished: mytool


2021-02-09 13:50:28,652 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:28,653 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


here we change the log-level to DEBUG:

In [14]:
try:
    tool.run(argv=['--log-level','DEBUG','--infile', str(GAMMA_FILE)])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2021-02-09 13:50:28,669 DEBUG [ctapipe.mytool] (application._config_changed): Config changed: {'AdvancedComponent': {'infile': '/github/home/.cache/ctapipe/gamma_test.simtel.gz'}, 'attach_subarray': <LazyConfigValue {}>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2021-02-09 13:50:28,673 INFO [ctapipe.mytool] (tool.initialize): ctapipe version 0.1.dev1+gceb4210


2021-02-09 13:50:28,673 INFO [ctapipe.mytool] (tool.run): Starting: mytool


2021-02-09 13:50:28,683 DEBUG [ctapipe.core.provenance] (provenance.start_activity): started activity: mytool


2021-02-09 13:50:28,686 DEBUG [ctapipe.core.traits] (traits.attach_subarray): argument '*' matched: ['LST_LST_LSTCam', 'MST_MST_NectarCam']


...

2021-02-09 13:50:29,010 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,010 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,011 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,013 DEBUG [ctapipe.mytool] (tool.write_provenance): PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "c1406e78-a0f5-4cae-9985-91dec5f598ef",
      "start": {
         "time_utc": "2021-02-09T13:50:27.793"
      },
      "stop": {
         "time_utc": "2021-02-09T13:50:28.308"
      },
      "system": {
         "ctapipe_version": "0.1.dev1+gceb4210",
         "ctapipe_resources_version": "not installed",
         "eventio_version": "1.5.1",
         "ctapipe_svc_path": null,
         "executable": "/usr/local/bin/python3",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "",
            "machine": "x86_64",
            "processor": "",
            "node": "f2daf0b42c94",
            "version": "#41~18.04.1-Ubuntu SMP Mon Jan 18 14:00:01 UTC 2021",
            "system": "Linux",
            "release": "5.4.0-1039-azure",
            "libcver": [
               "glibc",
               "2.28

2021-02-09 13:50:29,016 DEBUG [ctapipe.mytool] (application.exit): Exiting application: mytool


you can also set parameters directly in the class, rather than using the argument/configfile parser. This is useful if you are calling the Tool from a script rather than the command-line

In [15]:
tool.iterations = 1
tool.log_level = 0

try:
    tool.run(['--infile', str(GAMMA_FILE)])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2021-02-09 13:50:29,030 DEBUG [ctapipe.mytool] (application._config_changed): Config changed: {'AdvancedComponent': {'infile': '/github/home/.cache/ctapipe/gamma_test.simtel.gz'}, 'attach_subarray': <LazyConfigValue {}>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2021-02-09 13:50:29,034 DEBUG [ctapipe.mytool] (application._config_changed): Config changed: {'AdvancedComponent': {'infile': '/github/home/.cache/ctapipe/gamma_test.simtel.gz'}, 'attach_subarray': <LazyConfigValue {}>, 'MyTool': {'log_level': 'DEBUG', 'iterations': 3}}


2021-02-09 13:50:29,038 INFO [ctapipe.mytool] (tool.initialize): ctapipe version 0.1.dev1+gceb4210


2021-02-09 13:50:29,039 INFO [ctapipe.mytool] (tool.run): Starting: mytool


2021-02-09 13:50:29,049 DEBUG [ctapipe.core.provenance] (provenance.start_activity): started activity: mytool


...

2021-02-09 13:50:29,374 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,375 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,376 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,377 DEBUG [ctapipe.mytool] (tool.write_provenance): PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "c1406e78-a0f5-4cae-9985-91dec5f598ef",
      "start": {
         "time_utc": "2021-02-09T13:50:27.793"
      },
      "stop": {
         "time_utc": "2021-02-09T13:50:28.308"
      },
      "system": {
         "ctapipe_version": "0.1.dev1+gceb4210",
         "ctapipe_resources_version": "not installed",
         "eventio_version": "1.5.1",
         "ctapipe_svc_path": null,
         "executable": "/usr/local/bin/python3",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "",
            "machine": "x86_64",
            "processor": "",
            "node": "f2daf0b42c94",
            "version": "#41~18.04.1-Ubuntu SMP Mon Jan 18 14:00:01 UTC 2021",
            "system": "Linux",
            "release": "5.4.0-1039-azure",
            "libcver": [
               "glibc",
               "2.28

2021-02-09 13:50:29,380 DEBUG [ctapipe.mytool] (application.exit): Exiting application: mytool


see what happens when a value is set that is not of the correct type:

In [16]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

bad value: The 'iterations' trait of a MyTool instance expected an int, not the str 'badval'.


Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [17]:
tool.advanced.outfile = "Another.txt"

2021-02-09 13:50:29,397 WARNING [ctapipe.mytool.AdvancedComponent] (<ipython-input-1-a087b1827cdf>.on_outfile_changed): Outfile was changed to '{'name': 'outfile', 'old': None, 'new': PosixPath('/github/workspace/docs/examples/Another.txt'), 'owner': <__main__.AdvancedComponent object at 0x7ff84c2f8670>, 'type': 'change'}'


we see that the handler for `outfile` was called, and it receive a change dict that shows the old and new values.

create a tool using a config file:

In [18]:
tool2 = MyTool()

In [19]:
try:
    tool2.run(argv=['--config', 'Tools.json'])
except SystemExit as e:
    assert e.code == 0, f'Tool returned with error status {e}'

2021-02-09 13:50:29,420 INFO [ctapipe.mytool] (tool.initialize): ctapipe version 0.1.dev1+gceb4210


2021-02-09 13:50:29,421 INFO [ctapipe.mytool] (tool.run): Starting: mytool


2021-02-09 13:50:29,430 DEBUG [ctapipe.core.provenance] (provenance.start_activity): started activity: mytool


2021-02-09 13:50:29,433 DEBUG [ctapipe.core.traits] (traits.attach_subarray): argument '*' matched: ['LST_LST_LSTCam', 'MST_MST_NectarCam']


2021-02-09 13:50:29,436 DEBUG [ctapipe.mytool] (tool.run): CONFIG: {'MyTool': {'config_file': PosixPath('/github/workspace/docs/examples/Tools.json'), 'iterations': 5, 'log_config': {'version': 1, 'disable_existing_loggers': False, 'formatters': {'file': {'()': <class 'ctapipe.core.logging.PlainFormatter'>, 'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'}, 'console': {'()': <class 'ctapipe.core.logging.ColoredFormatter'>, 'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'}}, 'handlers': {'ctapipe-console': {'class': 'logging.StreamHandler', 'formatter': 'console', 'stream': 'ext://sys.stderr', 'level': 10}, 'console': {'class': 'logging.StreamHandler', 'formatter': 'console', 'stream': 'ext://sys.stderr', 'level': 'NOTSET'}}, 'loggers': {'ctapipe': {'level': 10, 'handlers': ['ctapipe-console'], 'propagate': False}}}, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_file': None, 'log_file_level': 'INFO', 'log_format': '[%(na

...

2021-02-09 13:50:29,959 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,961 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,963 INFO [ctapipe.mytool] (tool.write_provenance): Output: 


2021-02-09 13:50:29,965 DEBUG [ctapipe.mytool] (tool.write_provenance): PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "c1406e78-a0f5-4cae-9985-91dec5f598ef",
      "start": {
         "time_utc": "2021-02-09T13:50:27.793"
      },
      "stop": {
         "time_utc": "2021-02-09T13:50:28.308"
      },
      "system": {
         "ctapipe_version": "0.1.dev1+gceb4210",
         "ctapipe_resources_version": "not installed",
         "eventio_version": "1.5.1",
         "ctapipe_svc_path": null,
         "executable": "/usr/local/bin/python3",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "",
            "machine": "x86_64",
            "processor": "",
            "node": "f2daf0b42c94",
            "version": "#41~18.04.1-Ubuntu SMP Mon Jan 18 14:00:01 UTC 2021",
            "system": "Linux",
            "release": "5.4.0-1039-azure",
            "libcver": [
               "glibc",
               "2.28

2021-02-09 13:50:29,969 DEBUG [ctapipe.mytool] (application.exit): Exiting application: mytool


In [20]:
print(tool2.advanced.infile)

/github/workspace/docs/examples/something.txt


In [21]:
print(tool2.config)

{'MyTool': {'config_file': 'Tools.json', 'log_level': 'DEBUG'}, 'AdvancedComponent': {'infile': 'something.txt'}, 'attach_subarray': <LazyConfigValue {}>}


In [22]:
tool2.is_setup

True

In [23]:
tool3 = MyTool()

In [24]:
tool3.is_setup

False

In [25]:
tool3.initialize(argv=[])

In [26]:
tool3.is_setup

False

In [27]:
tool3

config_file,None,name of a configuration file with parameters to load in addition to command-line parameters (default: None)
iterations,5,Number of times to run (default: 5)
log_config,"{'version': 1, 'disable_existing_loggers': False, 'formatters': {'file': {'()': , 'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'}, 'console': {'()': , 'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'}}, 'handlers': {'ctapipe-console': {'class': 'logging.StreamHandler', 'formatter': 'console', 'stream': 'ext://sys.stderr', 'level': 30}, 'console': {'class': 'logging.StreamHandler', 'formatter': 'console', 'stream': 'ext://sys.stderr', 'level': 'NOTSET'}}, 'loggers': {'ctapipe': {'level': 30, 'handlers': ['ctapipe-console'], 'propagate': False}}}",(default: traitlets.Undefined)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_file,None,Filename for the log (default: None)
log_file_level,INFO,Logging Level for File Logging (default: INFO)
log_format,[%(name)s]%(highlevel)s %(message)s,The Logging format template (default: [%(name)s]%(highlevel)s %(message)s)
log_level,30,Set the log level by value or name. (default: 30)
provenance_log,/github/workspace/docs/examples/mytool.provenance.log,(default: None)
quiet,False,(default: False)
show_config,False,"Instead of starting the Application, dump configuration to stdout (default: False)"


In [28]:
tool

KeyError: 'MyComponent'

In [29]:
tool.comp2

value,-1,Value to use (default: -1)


## Getting the configuration of an instance

In [30]:
tool.get_current_config()

{'MyTool': {'config_file': None,
  'iterations': 3,
  'log_config': {'version': 1,
   'disable_existing_loggers': False,
   'formatters': {'file': {'()': ctapipe.core.logging.PlainFormatter,
     'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'},
    'console': {'()': ctapipe.core.logging.ColoredFormatter,
     'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'}},
   'handlers': {'ctapipe-console': {'class': 'logging.StreamHandler',
     'formatter': 'console',
     'stream': 'ext://sys.stderr',
     'level': 30},
    'console': {'class': 'logging.StreamHandler',
     'formatter': 'console',
     'stream': 'ext://sys.stderr',
     'level': 'NOTSET'}},
   'loggers': {'ctapipe': {'level': 30,
     'handlers': ['ctapipe-console'],
     'propagate': False}}},
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_file': None,
  'log_file_level': 'INFO',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message

In [31]:
tool.iterations = 12
tool.get_current_config()

{'MyTool': {'config_file': None,
  'iterations': 12,
  'log_config': {'version': 1,
   'disable_existing_loggers': False,
   'formatters': {'file': {'()': ctapipe.core.logging.PlainFormatter,
     'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'},
    'console': {'()': ctapipe.core.logging.ColoredFormatter,
     'fmt': '%(asctime)s %(levelname)s [%(name)s] (%(module)s.%(funcName)s): %(message)s'}},
   'handlers': {'ctapipe-console': {'class': 'logging.StreamHandler',
     'formatter': 'console',
     'stream': 'ext://sys.stderr',
     'level': 30},
    'console': {'class': 'logging.StreamHandler',
     'formatter': 'console',
     'stream': 'ext://sys.stderr',
     'level': 'NOTSET'}},
   'loggers': {'ctapipe': {'level': 30,
     'handlers': ['ctapipe-console'],
     'propagate': False}}},
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_file': None,
  'log_file_level': 'INFO',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(messag

## Writing a Sample Config File

In [32]:
print(tool.generate_config_file())

# Configuration file for mytool.

#------------------------------------------------------------------------------
# Application(SingletonConfigurable) configuration
#------------------------------------------------------------------------------
## This is an application.

## The date format used by logging formatters for %(asctime)s
#  Default: '%Y-%m-%d %H:%M:%S'
# c.Application.log_datefmt = '%Y-%m-%d %H:%M:%S'

## The Logging format template
#  Default: '[%(name)s]%(highlevel)s %(message)s'
# c.Application.log_format = '[%(name)s]%(highlevel)s %(message)s'

## Set the log level by value or name.
#  Choices: any of [0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL']
#  Default: 30
# c.Application.log_level = 30

## Instead of starting the Application, dump configuration to stdout
#  Default: False
# c.Application.show_config = False

## Instead of starting the Application, dump configuration to stdout (as JSON)
#  Default: False
# c.Application.show_config_json = Fa